<a href="https://colab.research.google.com/github/hylle77/BDS/blob/main/Group_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GROUP ASSIGNMENT 4 - ADVANCED AI APPLICATIONS

##Necessary imports and installments

In [ ]:
#Installments
!pip -q install langchain huggingface_hub google-search-results tiktoken chromadb lark langchain-together sentence_transformers gradio sentence_transformers -qqq

In [2]:
#Imports
from google.colab import userdata
import textwrap
import os
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from sentence_transformers import SentenceTransformer
from langchain_together import Together
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
import gradio as gr

#Loading Togehter.ai API
os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY')

#Creating Embeddings using Hugginface
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-bert-base-dot-v5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Loading the data

AI-Generated dataset with metadata attached

In [3]:
#Creating a dataset
docs = [
    Document(
        page_content="Rich and complex with notes of caramel, chocolate, and a hint of coffee. Balanced bitterness.",
        metadata={"name": "Bourbon County Stout", "year": 2020, "rating": 98, "style": "Imperial Stout", "ABV": 14.2, "IBU": 60, "country": "USA", "price_per_bottle": 20},
    ),
    Document(
        page_content="Bright and citrusy, with a crisp finish. Notes of grapefruit and pine.",
        metadata={"name": "Sculpin IPA", "year": 2021, "rating": 95, "style": "IPA", "ABV": 7.0, "IBU": 70, "country": "USA", "price_per_bottle": 15},
    ),
    Document(
        page_content="Smooth and malty, with layers of toffee and nuts. Subtle hop finish.",
        metadata={"name": "Celebrator Doppelbock", "year": 2021, "rating": 97, "style": "Doppelbock", "ABV": 6.7, "IBU": 24, "country": "Germany", "price_per_bottle": 12},
    ),
    Document(
        page_content="Light and refreshing, with a delicate balance of sweetness and acidity. Notes of apple and pear.",
        metadata={"name": "Weihenstephaner Hefeweissbier", "year": 2021, "rating": 96, "style": "Hefeweizen", "ABV": 5.4, "IBU": 14, "country": "Germany", "price_per_bottle": 10},
    ),
    Document(
        page_content="Dark and intense, with rich fruit flavors and a robust body. Complex aroma of dark fruits and spice.",
        metadata={"name": "Rochefort 10", "year": 2020, "rating": 99, "style": "Trappist Quadrupel", "ABV": 11.3, "IBU": 27, "country": "Belgium", "price_per_bottle": 18},
    ),
Document(
        page_content="Aromatic with a rich hoppy flavor and notes of tropical fruit. Crisp and refreshing.",
        metadata={"name": "Ghost in the Machine", "year": 2021, "rating": 97, "style": "Double IPA", "ABV": 8.5, "IBU": 70, "country": "USA", "price_per_bottle": 16},
    ),
    Document(
        page_content="Full-bodied with a velvety texture and hints of vanilla and bourbon. Warming alcohol finish.",
        metadata={"name": "Dragon's Milk", "year": 2020, "rating": 94, "style": "Bourbon Barrel-Aged Stout", "ABV": 11.0, "IBU": 31, "country": "USA", "price_per_bottle": 18},
    ),
    Document(
        page_content="Bright yellow with a frothy head, offering a spicy coriander and orange peel aroma. Light and zesty.",
        metadata={"name": "Hoegaarden", "year": 2021, "rating": 92, "style": "Witbier", "ABV": 4.9, "IBU": 15, "country": "Belgium", "price_per_bottle": 9},
    ),
    Document(
        page_content="Pours a deep amber with a rich malt profile, balanced by a pronounced hop bitterness. Caramel and floral notes.",
        metadata={"name": "Sierra Nevada Celebration", "year": 2021, "rating": 95, "style": "American IPA", "ABV": 6.8, "IBU": 65, "country": "USA", "price_per_bottle": 12},
    ),
    Document(
        page_content="Dark, roasty flavors with hints of chocolate and coffee. Smooth and creamy with a bitter finish.",
        metadata={"name": "Guinness Draught", "year": 2021, "rating": 93, "style": "Irish Dry Stout", "ABV": 4.2, "IBU": 45, "country": "Ireland", "price_per_bottle": 8},
    ),
    Document(
        page_content="Sour and salty with a lemon tartness and a crisp, dry finish. Refreshing and light.",
        metadata={"name": "Gose Gone Wild", "year": 2020, "rating": 96, "style": "Gose", "ABV": 4.3, "IBU": 10, "country": "Germany", "price_per_bottle": 14},
    ),
    Document(
        page_content="Fruity and spicy with notes of banana and clove. Effervescent and golden, with a sweet, bready malt backbone.",
        metadata={"name": "Paulaner Hefe-Weissbier", "year": 2021, "rating": 94, "style": "Hefeweizen", "ABV": 5.5, "IBU": 12, "country": "Germany", "price_per_bottle": 11},
    ),
    Document(
        page_content="A unique blend of hops gives a floral aroma, with a balanced malt sweetness and a touch of citrus.",
        metadata={"name": "Punk IPA", "year": 2021, "rating": 90, "style": "IPA", "ABV": 5.6, "IBU": 45, "country": "Scotland", "price_per_bottle": 13},
    ),
    Document(
        page_content="Golden and clear with a delicate balance of bitterness and malt sweetness. Crisp finish.",
        metadata={"name": "Pilsner Urquell", "year": 2021, "rating": 89, "style": "Pilsner", "ABV": 4.4, "IBU": 40, "country": "Czech Republic", "price_per_bottle": 10},
    ),
    Document(
        page_content="Deep ruby in color with a complex flavor profile of dark fruits, caramel, and toffee. Warm alcohol notes.",
        metadata={"name": "Old Rasputin", "year": 2020, "rating": 98, "style": "Russian Imperial Stout", "ABV": 9.0, "IBU": 75, "country": "USA", "price_per_bottle": 12},
    )
]

##Storing the Embeddings in Chroma

In [4]:
#Get the craft beers in the store
vectorstore = Chroma.from_documents(docs, embeddings)

## Creating self-querying retriever

In [5]:
#Define metadata schema for the data
metadata_field_info = [
    AttributeInfo(
        name="style",
        description="The style of the beer",
        type="string",
    ),
    AttributeInfo(
        name="name",
        description="The name of the beer",
        type="string",
    ),
    AttributeInfo(
        name="ABV",
        description="Alcohol by Volume, a percentage representing the concentration of alcohol in the beer",
        type="float",
    ),
    AttributeInfo(
        name="IBU",
        description="International Bitterness Units, a measure of the bitterness of the beer",
        type="integer",
    ),
    AttributeInfo(
        name="year",
        description="The year the beer was released",
        type="integer",
    ),
    AttributeInfo(
        name="country",
        description="The name of the country the beer comes from",
        type="string",
    ),
    AttributeInfo(
        name="rating",
        description="The rating of the beer, typically on a scale from 0 to 100",
        type="integer",
    ),
    AttributeInfo(
        name="price_per_bottle",
        description="Average price per bottle in EUR",
        type="float",
    ),
]
document_content_description = "Brief description of the beer"

In [6]:
#Initializing the model from Together
llm = Together(
    model="teknium/OpenHermes-2p5-Mistral-7B",
    temperature=0.3,
    max_tokens=256,
    top_k=50,
    # together_api_key="..."
)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [7]:
#Example specifying a relevant query
retriever.get_relevant_documents("Name me some beers?")

[Document(page_content='Sour and salty with a lemon tartness and a crisp, dry finish. Refreshing and light.', metadata={'ABV': 4.3, 'IBU': 10, 'country': 'Germany', 'name': 'Gose Gone Wild', 'price_per_bottle': 14, 'rating': 96, 'style': 'Gose', 'year': 2020}),
 Document(page_content='Aromatic with a rich hoppy flavor and notes of tropical fruit. Crisp and refreshing.', metadata={'ABV': 8.5, 'IBU': 70, 'country': 'USA', 'name': 'Ghost in the Machine', 'price_per_bottle': 16, 'rating': 97, 'style': 'Double IPA', 'year': 2021}),
 Document(page_content='Bright and citrusy, with a crisp finish. Notes of grapefruit and pine.', metadata={'ABV': 7.0, 'IBU': 70, 'country': 'USA', 'name': 'Sculpin IPA', 'price_per_bottle': 15, 'rating': 95, 'style': 'IPA', 'year': 2021}),
 Document(page_content='Dark and intense, with rich fruit flavors and a robust body. Complex aroma of dark fruits and spice.', metadata={'ABV': 11.3, 'IBU': 27, 'country': 'Belgium', 'name': 'Rochefort 10', 'price_per_bottle':

In [8]:
#Example specifying a query and a filter
retriever.get_relevant_documents("I want a beer that has fruity notes and has a rating above 97")

[Document(page_content='Aromatic with a rich hoppy flavor and notes of tropical fruit. Crisp and refreshing.', metadata={'ABV': 8.5, 'IBU': 70, 'country': 'USA', 'name': 'Ghost in the Machine', 'price_per_bottle': 16, 'rating': 97, 'style': 'Double IPA', 'year': 2021}),
 Document(page_content='Smooth and malty, with layers of toffee and nuts. Subtle hop finish.', metadata={'ABV': 6.7, 'IBU': 24, 'country': 'Germany', 'name': 'Celebrator Doppelbock', 'price_per_bottle': 12, 'rating': 97, 'style': 'Doppelbock', 'year': 2021})]

##Exploring diffrent prompt engineering techniques

In [9]:
# N-shot Prompting Template
n_shot_template = """You are a beer expert. Here are some examples of beer recommendations:
Example 1:
Question: Recommend a beer for a summer barbecue.
Answer: A light lager like Summerfest from Sierra Nevada is perfect for hot days.

Example 2:
Question: What's a good beer with spicy food?
Answer: The Sculpin IPA from Ballast Point has a bold, citrus flavor that complements spicy dishes well.

Now, your question:
{question}
"""
prompt_n_shot = ChatPromptTemplate.from_template(n_shot_template)

# Use the modified template with examples
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_n_shot
    | llm
    | StrOutputParser()
)

input_query = "Recommend a craft beer with fruity notes and an ABV greater than 6%."
output = chain.invoke(input_query)
print(textwrap.fill(output))

An't no sunshine IPA from Green Flash Brewing Co. is a great choice.
It has a citrusy, tropical fruit flavor and an ABV of 7.5%.


In [10]:
#CoT Prompting Template
cot_template = """You are a beer expert. Think step by step to answer the question:
{context}

Question: {question}
Think First: Consider the beer's flavor profile and how it might pair with foods or occasions.
Next Step: Reflect on the ABV as it contributes to the beer's body and overall experience.
Final Recommendation:
"""
prompt_cot = ChatPromptTemplate.from_template(cot_template)

# Use the CoT template
chain2 = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_cot
    | llm
    | StrOutputParser()
)

input_query = "What beer goes well with grilled salmon?"
output = chain2.invoke(input_query)
print(textwrap.fill(output))

Based on the provided information, the beer that would go well with
grilled salmon is "Gose Gone Wild". This beer has a sour and salty
flavor with a lemon tartness and a crisp, dry finish. Its refreshing
and light taste would complement the rich flavors of grilled salmon.


In [11]:
#Generated Knowledge Prompting Template
gen_knowledge_template = """You are a beer expert. Generate useful insights about craft beers, then answer the question:
Generate Insight: Describe the key characteristics of a great craft beer focusing on innovative brewing techniques and unique ingredients.

Now, use this insight to answer:
{question}
"""
prompt_gen_knowledge = ChatPromptTemplate.from_template(gen_knowledge_template)

# Use the generated knowledge template
chain3 = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_gen_knowledge
    | llm
    | StrOutputParser()
)

input_query = "What makes an IPA stand out from other beer styles?"
output = chain3.invoke(input_query)
print(textwrap.fill(output))

 AI: A great craft beer is characterized by its innovative brewing
techniques and unique ingredients. These factors contribute to the
distinct flavor profile and aroma of the beer. When it comes to IPAs,
they stand out from other beer styles due to their high hop content,
which imparts a bitter and citrusy flavor. The use of different hop
varieties and brewing methods can also create variations in the aroma
and taste of an IPA, making it unique and memorable.


##Gradio depolyment

In [15]:
#Defining function for Gradio
def recommend_beers(question, technique='n-shot'):
    if technique == 'n-shot':
        output = chain.invoke(question)
    elif technique == 'cot':
        output = chain2.invoke(question)
    elif technique == 'gen-knowledge':
        output = chain3.invoke(question)
    else:
        output = "Technique not recognized. Please choose 'N-shot', 'CoT', or 'Gen-Knowledge'."
    return output

interface = gr.Interface(
    fn=lambda question, technique: recommend_beers(question, technique),
    inputs=[gr.Textbox(label="Your Question"), gr.Dropdown(['n-shot', 'cot', 'gen-knowledge'], label="Prompting Technique")],
    outputs=gr.Textbox(),
    title="Beer Recommendation Expert",
    description="Ask me for craft beer recommendations or information about craft beers! Select the prompting technique you'd like to use.",
)

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2560fe4120ab3ceb86.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
